# Ag-Analytics® - Farmland Finder -  Land Value Service API 

Ag-Analytics® Land Value Service API provides users with easy and fast access to filter and find land for sale, complete with the sales and geographic data components to get a detailed description of that land’s value. The data of this service is originally provided by [Farmland Finder](https://www.farmlandfinder.com). In this version of Land Value Service API, each parcel/land has a full description of the entire sale/transaction. Multiple parcel/land record can belong to one transaction. In geographic attribute, each result record gives the detailed description of a certain land/parcel itself, while the attribute in ‘sales’ may carry the information for the transaction/sale other than this parcel/land.

### Required libraries

In [33]:
import requests
import json
import pandas as pd
import folium
import mplleaflet
import geojson
import shapely.wkt

### Request Parameter Details

Request URL:  https://ag-analytics.azure-api.net/landvalue-v2

       
1). __Location Parameters__(Required): 
    To make a valid request,at least one location parameters need to be provided.
    
    i. State:  The name of State in string format.Title cased.
    
    ii. County: The name of State in string format. Title cased.This parameter will be valid only if the State parameter is provided
    
    iii.Bounding_box: Area of interest in geoJSON format(See example below).
    

2). __Sale Condition Parameters__(optional): Optional parameters to specify response based on sale conditions

    i. Status: Sale Condition of the property. Vaild options are: Sold, For Sale, Expired Listing.
    
    ii. StartDate (Required only if Status is Sold):Searching starting date of the property Sale Date. In format 'yyyy-mm-dd'
    
    iii.EndDate (Required only if Status is Sold):Searching end date of the property Sale Date. In format 'yyyy-mm-dd'

### Response Description

__Basic Information__

1).ID :The unique sale id for each parcel as a string

2).Shape: The boundary of the parcel/property in Well Known Text type

3).State: The state where the parcel/property locates in. 

4).County:The county where the parcel/property locates in.

5).Entry_Updated:The date of the sales information has been updated

__Sale Information__

1).Avg_CSR2: the average The Iowa Corn Suitability Rating(Soil Productivity Index)

2).CRP: If the property joined the Conservation Reserve Program.(‘Yes’ or ‘No’)

3).Total_Acres: the total acres of the entire sale

4).Tillable_Acres: the tillable acres of the sale

5).Percent_Tillable: the percent tillable area

6).CRP_Acres: the CRP acres

7).Sale_Price: the total sale price of the sale record

8).Price_Acre: the price per acre of the sale record

9).Status: one of the following values: "For Sale", "Listing Expired", "Sold"

10).Sale_Condition: one of the following values: "Auction", "Listing"

11).Listing_Agent: the listing agent name

12).Buyer: the buyer name as a string

13).Sale_Date: The sale date string in YYYY-MM-DD format. (When the parcel is still listing, the attribute will be Null

14).Taxes_total: taxes for the sale as a float

15).Assessed_Land: If the land/parcel has been assessed

16).Broker_URL: The URL link to the broker listing webpage as a string


__Geographic Information__

1).lat_center: the latitudinal center of the parcel/property as a float value

2).lng_center: the longitudinal center of the parcel/property as a float value

3).range: the range as a string (ex: 26W - always include E or W)

4).sect: the section as string (ex: 17)

5).twnshp: the township as a string (ex: 78N - always include N or S)

6).county_name: the county name as a string (Title cased - for ex: Osceola, Polk, etc.)

7).state_name: the state name as a string (Title cased - for ex: Michigan)

8).STATEFP: the FIPS to state level as a string

9).FIPS: the FIPS to county level as a string


### Request Parameter Example

In [37]:
State= "Michigan"
County= "Houghton"
Bounding_box = '{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-88.6,47.04],[-88.54,47.04],[-88.54,47.1],[-88.6,47.1],[-88.6,47.04]]]}}'

Status = 'Sold'
StartDate = '2019-08-08'
EndDate = '2020-01-01'

#Format the inputs()
values = {
    'State':State,
    'County' : County,
    'Bounding_Box': Bounding_box,
    'StartDate':StartDate,
    'EndDate':EndDate,
    'Status' :Status
}

# Header for using a subscription key.
headers = {'Content-Type': 'application/x-www-form-urlencoded','Ocp-Apim-Subscription-Key': "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"}

### API Function

In [35]:
def LandValueService(values,headers):
    try:
        url = 'https://ag-analytics.azure-api.net/landvalue-v2'
        
        response = requests.post(url, data = values,headers = headers).json()
        #resp = json.loads(response)
        print(response)
        
        return response
    
    except Exception as e:
        print(e)
        raise e

### Calling API Function and Displaying Response

In [38]:
LandValueResponse = LandValueService(values)

[{'ID': '5e277282526a87486f9d3a74', 'Entry_Updated': '0019-08-01 00:00:00', 'basics': {'Shape': 'POLYGON((-88.5901467200157 47.0475234037846, -88.5965618942772 47.0475663534384, -88.5962213412716 47.0463376577456, -88.595880830534 47.0451090572013, -88.5955141283121 47.0437858907245, -88.5948444899643 47.0414058363943, -88.5944499955406 47.0399963668948, -88.5893904796341 47.039925019316, -88.5843330336802 47.0398535058385, -88.5846001922596 47.0436668695639, -88.5848663176613 47.0474801244352, -88.5901467200157 47.0475234037846))', 'State': 'Michigan', 'County': 'Houghton'}, 'sales': {'Avg_CSR2': 21.0952, 'CRP': None, 'Total_Acres': 177.0, 'Tillable_Acres': 0.0, 'Percent_Tillable': 0.0, 'CRP_Acres': None, 'Sale_Price': '200000.0000', 'Price_Acre': '1051.0000', 'Status': 'For Sale', 'Sale_Condition': 'Listing', 'Listing_Agent': 'Northern Michigan Land Brokers', 'Buyer': None, 'Sale_Date': None, 'Taxes_total': None, 'Assessed_Land': None, 'Broker_URL': 'http://www.joshsteinland.com/list

### Response by Category

In [22]:
#Format response to dataframes
ID = []
basics = []
sales = []
geographic =[]
for record in LandValueResponse:
    basics.append(record['basics'])
    sales.append(record['sales'])
    geographic.append(record['geographic'])
    ID.append(record['ID'])
basics_db = pd.DataFrame(basics)
basics_db['ID'] = ID
sales_db = pd.DataFrame(sales)
sales_db['ID'] = ID
geographic_db = pd.DataFrame(geographic)
geographic_db['ID'] = ID

#### 1.Parcel Basics 

In [26]:
#Narrow table
basics_db.set_index('ID').T
#Wide table
# basics_db.set_index('ID')

ID,5e277282526a87486f9d3a74,5e277282526a87486f9d3a20
County,Houghton,Houghton
Shape,"POLYGON((-88.5901467200157 47.0475234037846, -...","POLYGON((-88.5594344373249 47.0924597366254, -..."
State,Michigan,Michigan


#### 2.Sale Information

In [27]:
sales_db.set_index('ID').T

ID,5e277282526a87486f9d3a74,5e277282526a87486f9d3a20
Assessed_Land,None,None
Avg_CSR2,21.0952,15.9878
Broker_URL,http://www.joshsteinland.com/listing.php?targe...,http://www.joshsteinland.com/listing.php?targe...
Buyer,None,None
CRP,None,None
CRP_Acres,None,None
Listing_Agent,Northern Michigan Land Brokers,Northern Michigan Land Brokers
Percent_Tillable,0,1
Price_Acre,1051.0000,2503.0000
Sale_Condition,Listing,Listing


#### 3.Location Information

In [28]:
geographic_db.set_index('ID').T

ID,5e277282526a87486f9d3a74,5e277282526a87486f9d3a20
FIPS,26061,26061
STATEFP,26,26
county_name,Houghton,Houghton
lat_center,47.0438,47.0919
lng_center,-88.5901,-88.5605
range,34W,34W
sect,26,12
state_name,Michigan,Michigan
twnshp,54N,54N


### Visualize Parcels on Map

In [29]:
#Convert shapes to geojson
shapes = []
for record in LandValueResponse:
    wkt = record['basics']['Shape']
    g1 = shapely.wkt.loads(wkt)
    g2 = geojson.Feature(geometry=g1)
    shapes.append(g2)

In [30]:
#Display parcels on map
zoom_point=json.loads(Bounding_box)['geometry']['coordinates'][0][0]
m = folium.Map([zoom_point[1],zoom_point[0]],tiles='Cartodb Positron', zoom_start=11,width='70%', height='100%')
for shape in shapes:
    folium.GeoJson(shape).add_to(m)
m